In [1]:
#import needed package
import asyncio
import httpx
import pandas as pd
import secret
import refresh
from time import perf_counter

In [2]:
#authtokens and headers
id_token = refresh.refreshed()
headers = {"content-type":"application/json", "x-api-key": secret.api_key, "id-token": id_token}

In [3]:
#scrape all
vinny_id = 62869

In [4]:
#api calls
sem = asyncio.Semaphore(100)
#scrape all id

async def get_result(vinny_id:int):
    async with sem:
        async with httpx.AsyncClient() as client:
            r = await client.get(f'https://bps-world.vincere.io/api/v2/candidate/{vinny_id}/subindustries', headers=headers)
            if r.status_code == 200:
                data = [{**d, 'vinny_id': vinny_id} for d in r.json()]
                results.append(data)
                vincere.append(vinny_id)
                return
            else:
                return

#trigger main function
async def main():
    tasks = []
    vinny_id = 62869
    #gen_id = df['vinny_id']
    while vinny_id <= 103739:
        tasks.append(get_result(vinny_id))
        vinny_id += 1
#     tasks = []
#     vinny_id = 85738
#     tasks.append(get_result(vinny_id))
    await asyncio.gather(*tasks, return_exceptions=True)

#future container for finding errors and stuff
results = []

In [5]:
#trigger the whole function
start = perf_counter()
await main()
stop = perf_counter()
print('time taken:', stop - start, ' seconds')

time taken: 1392.1066551  seconds


In [6]:
flat_data = [d for sublist in results for d in sublist]

In [8]:
df = pd.DataFrame(flat_data, columns=['id', 'name', 'vinny_id'])

In [9]:
df.to_excel('vincere_sub_industry_candidate_230223.xlsx')